In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import pyodbc
from datetime import datetime

# ==============================
# 1. เชื่อม SQL Server
# ==============================
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=LAPTOP-HJRRM85F;"
    "Database=FoodRecommendDB;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

# ==============================
# 2. โหลดตารางจาก DB
# ==============================
users = pd.read_sql("SELECT * FROM User_2", conn)
menu_lookup = pd.read_sql("SELECT * FROM menus_2", conn)
df_final = pd.read_csv("df_final2.csv")  
user_ratings = pd.read_sql("SELECT * FROM Ratings", conn)
disease_prep = pd.read_csv(r"preposition/disease_prep.csv")  # โหลด health constraints

# normalize columns
df_final.columns = df_final.columns.str.lower()
menu_lookup.columns = menu_lookup.columns.str.lower()
user_ratings.columns = user_ratings.columns.str.lower()
users.columns = users.columns.str.lower()

# ==============================
# 3. ฟังก์ชันช่วยคำนวณ
# ==============================
def calculate_age(birthdate):
    if pd.isna(birthdate):
        return None
    today = datetime.today()
    return today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))

def calculate_bmi(weight, height_cm):
    """
    คำนวณ BMI (Body Mass Index)
    weight: น้ำหนัก (kg)
    height_cm: ความสูง (cm)
    return: BMI (float, 2 ตำแหน่ง) หรือ None ถ้าข้อมูลไม่ครบ
    """
    if weight is None or height_cm is None or height_cm == 0:
        return None
    height_m = height_cm / 100
    bmi = weight / (height_m ** 2)
    return round(bmi, 2)

def calculate_bmr(weight, height_cm, age, gender):
    """
    คำนวณ BMR (Basal Metabolic Rate) โดยใช้สูตร Harris-Benedict
    weight: น้ำหนัก (kg)
    height_cm: ความสูง (cm)
    age: อายุ (ปี)
    gender: 'male' หรือ 'female'
    return: BMR (float, 2 ตำแหน่ง) หรือ None ถ้าข้อมูลไม่ครบ
    """
    if None in [weight, height_cm, age] or height_cm == 0:
        return None
    
    gender = gender.lower() if gender else ''
    
    if gender == 'male':
        bmr = 88.36 + (13.4 * weight) + (4.8 * height_cm) - (5.7 * age)
    else:
        bmr = 447.6 + (9.25 * weight) + (3.1 * height_cm) - (4.33 * age)
    
    return round(bmr, 2)


def get_activity_factor(user_id, conn):
    """
    ดึง activity_factor ของ user จากตาราง activities ตาม activity_id ของ user
    """
    query = f"""
        SELECT a.activity_factor
        FROM user_2 u
        JOIN activities a ON u.activity_id = a.activity_id
        WHERE u.user_id = {user_id}
    """
    df = pd.read_sql(query, conn)
    if df.empty:
        return 1.55  # ค่า default ถ้าไม่มีข้อมูล
    return df.iloc[0]['activity_factor']

def calculate_tdee(user_id, bmr, conn):
    """
    คำนวณ TDEE โดยใช้ BMR และ activity_factor ของ user
    """
    if bmr is None:
        return None
    activity_factor = get_activity_factor(user_id, conn)
    return round(bmr * activity_factor, 2)


def get_latest_mood(user_id):
    query = f"""
        SELECT cu.user_id, m.moods
        FROM CurrentUserMood cu
        JOIN Mood m ON cu.mood_id = m.mood_id
        WHERE cu.user_id = {user_id}
        ORDER BY cu.timestamp DESC
    """
    mood_df = pd.read_sql(query, conn)
    if mood_df.empty:
        return None
    return mood_df.iloc[0]['moods'].lower()



# ==============================
# 4. ฟังก์ชัน health constraints
# ==============================

target_diseases = [
    "diabetes", "hypertension", "gout", "obesity", "kidney",
    "dyslipidemia", "heart_disease", "celiac", "lactose_intolerance",
    "anemia", "hyperuricemia"
]

def filter_by_health_constraints(user_id, df_final, df_menu_lookup):
    user_df = users[users['user_id'] == user_id]
    if user_df.empty:
        print(f"⚠️ User ID {user_id} ไม่มีในฐานข้อมูล")
        return df_final
    user = user_df.iloc[0]

    user_diseases = [d for d in target_diseases if d in str(user['disease']).split(',')]

    if not user_diseases:
        return df_final[["food_id"]]

    nutrient_cols = ['food_id','calories','protein','fat','carbs','sugar','sodium','cholesterol','potassium','saturated_fat']
    df_nutrients = df_menu_lookup[nutrient_cols].copy()
    df_nutrients.columns = [c.lower() for c in df_nutrients.columns]

    df_filtered = df_final[['food_id']].drop_duplicates().merge(df_nutrients, on='food_id', how='left')

    for disease in user_diseases:
        disease_df = disease_prep[disease_prep['disease'] == disease]
        if disease_df.empty:
            print(f"⚠️ ไม่มีข้อมูลโรค {disease} ใน disease_prep")
            continue
        disease_row = disease_df.iloc[0]

        avoid_nutrients = str(disease_row['avoid_nutrients']).split(',')
        for nutrient in avoid_nutrients:
            nutrient = nutrient.strip().lower()
            if nutrient and nutrient != "none" and nutrient in df_filtered.columns:
                if nutrient == "sugar":
                    df_filtered = df_filtered[df_filtered[nutrient] <= 10]
                elif nutrient == "sodium":
                    df_filtered = df_filtered[df_filtered[nutrient] <= 400]
                elif nutrient == "fat":
                    df_filtered = df_filtered[df_filtered[nutrient] <= 20]
                elif nutrient == "calories":
                    df_filtered = df_filtered[df_filtered[nutrient] <= 600]
                elif nutrient == "cholesterol":
                    df_filtered = df_filtered[df_filtered[nutrient] <= 100]
                elif nutrient == "potassium":
                    df_filtered = df_filtered[df_filtered[nutrient] <= 400]
                elif nutrient == "protein":
                    df_filtered = df_filtered[df_filtered[nutrient] <= 20]
                elif nutrient == "saturated_fat":
                    df_filtered = df_filtered[df_filtered[nutrient] <= 10]

        avoid_ingredients = str(disease_row['avoid_ingredients']).split(',')
        for ing in avoid_ingredients:
            ing = ing.strip()
            if ing and ing != "none" and ing in df_final.columns:
                df_filtered = df_filtered[df_final[ing] <= 0.1]

    return df_filtered[["food_id"]]

# ==============================
# 5. ฟังก์ชัน Recommendation
# ==============================
def content_based_recommend(user_id, top_n=5):
    user_df = users[users['user_id'] == user_id]
    if user_df.empty:
        print(f"⚠️ User ID {user_id} ไม่มีในฐานข้อมูล")
        return df_final
    user = user_df.iloc[0]

    # คำนวณ bmi/bmr/tdee
    age = calculate_age(user['birthdate'] if 'birthdate' in user else user['BirthDate'])
    bmi = calculate_bmi(user['weight'], user['height'])
    bmr = calculate_bmr(user['weight'], user['height'], age, user['gender'])
    tdee = calculate_tdee(uid, bmr, conn)
    current_mood = get_latest_mood(user_id)

    # กรอง diet_type
    if user['diet_type'] == 'veg':
        df_filtered = df_final[df_final['veg_non_veg'] == 1].copy()
    else:
        df_filtered = df_final.copy()

    # กรอง dislikes
    dislikes = str(user['dislikes']).split(',')
    for ing in dislikes:
        ing = ing.strip()
        if ing in df_filtered.columns:
            df_filtered = df_filtered[df_filtered[ing] == 0]

    # กรองตาม health constraints
    allowed_foods = set(filter_by_health_constraints(user_id, df_final, menu_lookup)["food_id"].unique())
    df_filtered = df_filtered[df_filtered['food_id'].isin(allowed_foods)]

    # Features
    features = [c for c in df_filtered.columns if c not in ["food_id","food_name"]]
    df_filtered[features] = df_filtered[features].fillna(0)

    # User vector
    user_vector = np.zeros(len(features))
    if "calories" in features:
        tdee_val = tdee if tdee is not None else 2000  # ค่า default 2000 kcal
        user_vector[features.index("calories")] = tdee_val * 0.9
    if "protein" in features:
        user_vector[features.index("protein")] = 20 if bmi >= 25 else 15
    if "fat" in features:
        user_vector[features.index("fat")] = 20 if bmi < 18.5 else 10

    # Mood
    mood_cols = [col for col in features if col in ["happy","neutral","stressed","sad"]]
    for mc in mood_cols:
        user_vector[features.index(mc)] = 0
    if current_mood in mood_cols:
        user_vector[features.index(current_mood)] = 1

    # Likes
    likes = str(user['likes']).split(',')
    for like in likes:
        like = like.strip()
        if like in features:
            user_vector[features.index(like)] = 1

    similarity = cosine_similarity(df_filtered[features].values, [user_vector])
    top_indices = np.argsort(similarity[:,0])[::-1][:top_n]

    food_ids = df_filtered.iloc[top_indices]['food_id'].values
    result = menu_lookup[menu_lookup['food_id'].isin(food_ids)][
        ["food_id","food_name","category","calories","protein","carbs","fat","sugar","sodium"]
    ]
    result = result.set_index('food_id').reindex(food_ids).reset_index()

    return result

def collaborative_recommend(user_id, top_n=5, include_eaten=True):
    if user_ratings.empty:
        return pd.DataFrame()

    reader = Reader(rating_scale=(1, 10))
    data = Dataset.load_from_df(user_ratings[['user_id', 'food_id', 'rating']], reader)
    trainset = data.build_full_trainset()
    algo = SVD(random_state=42)
    algo.fit(trainset)

    allowed_foods = set(filter_by_health_constraints(user_id, df_final, menu_lookup)["food_id"].unique())
    all_foods = set(menu_lookup['food_id'].unique())
    eaten_foods = set(user_ratings[user_ratings['user_id'] == user_id]['food_id'].unique())

    if include_eaten:
        candidate_foods = sorted(list(all_foods & allowed_foods))
    else:
        candidate_foods = sorted(list((all_foods - eaten_foods) & allowed_foods))

    if not candidate_foods:
        return pd.DataFrame()

    predictions = [algo.predict(user_id, fid) for fid in candidate_foods]
    predictions.sort(key=lambda x: x.est, reverse=True)

    top_pred = predictions[:top_n]
    food_ids = [int(pred.iid) for pred in top_pred]

    result = menu_lookup[menu_lookup['food_id'].isin(food_ids)][
        ["food_id", "food_name", "category","calories", "protein", "carbs", "fat", "sugar"]
    ]
    result['pred_score'] = [pred.est for pred in top_pred]
    result = result.set_index('food_id').reindex(food_ids).reset_index()

    return result

def hybrid_recommend(user_id, top_n=5, alpha=0.5):
    content_rec = content_based_recommend(user_id, top_n=20)
    collab_rec = collaborative_recommend(user_id, top_n=20)

    if collab_rec.empty:
        return content_rec.head(top_n)[["food_id","food_name","category", "calories", "protein", "carbs", "fat","sugar","sodium"]]

    content_rec = content_rec.copy().sort_values("food_id")
    content_rec["content_score"] = np.arange(len(content_rec), 0, -1)

    collab_rec = collab_rec.copy().sort_values("food_id")
    collab_rec["collab_score"] = np.arange(len(collab_rec), 0, -1)

    merged = pd.concat([content_rec, collab_rec], ignore_index=True)
    merged = merged.groupby("food_name", as_index=False).first()
    merged["content_score"] = merged.get("content_score", 0)
    merged["collab_score"] = merged.get("collab_score", 0)
    merged["hybrid_score"] = alpha * merged["content_score"] + (1 - alpha) * merged["collab_score"]

    return merged.sort_values("hybrid_score", ascending=False).head(top_n)[
        ["food_id","food_name","category", "calories", "protein", "carbs", "fat","sugar",'sodium']
    ]

# ==============================
# 6. ทดสอบระบบ
# ==============================
uid = 2
user_df = users[users["user_id"]==uid].copy()
if not user_df.empty:
    user = user_df.iloc[0]
    age = calculate_age(user['birthdate'] if 'birthdate' in user else user['BirthDate'])
    bmi = calculate_bmi(user['weight'], user['height'])
    bmr = calculate_bmr(user['weight'], user['height'], age, user['gender'])
    tdee = calculate_tdee(uid, bmr, conn)

    current_mood = get_latest_mood(uid)

    user_info = pd.DataFrame([{
        "username": user["username"],
        "gender": user["gender"],
        "age": age,
        "bmi": bmi,
        "bmr": bmr,
        "tdee": tdee,
        "disease": user["disease"],
        "current_mood": current_mood
    }])
    print("👤 User Information:")
    print(user_info.to_string(index=False))

print("\n🎯 Hybrid Recommendation:")
print(hybrid_recommend(uid, top_n=5))


👤 User Information:
username gender  age   bmi     bmr    tdee  disease current_mood
     Tom   male   30 22.86 1695.36 2034.43 diabetes      neutral

🎯 Hybrid Recommendation:
   food_id                                 food_name   category  calories  \
7       47  almond and chicken momos (without shell)    Chinese     220.0   
3      315                              Summer Rolls  Vietnames     150.0   
0      312                                Lamb Tikka     Indian     320.0   
1      332                             Mushroom Rice   Nepalese     220.0   
2      373  Stir-Fried Lettuces With Crispy Shallots    Chinese     150.0   

   protein  carbs   fat  sugar  sodium  
7     18.0   25.0   8.0    3.0   450.0  
3      4.0   25.0   2.0    3.0   150.0  
0     28.0   20.0  18.0    3.0   600.0  
1      6.0   32.0   8.0    3.0   200.0  
2      3.0   15.0   5.0    3.0   200.0  


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13452\88213119.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users = pd.read_sql("SELECT * FROM User_2", conn)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13452\88213119.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  menu_lookup = pd.read_sql("SELECT * FROM menus_2", conn)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13452\88213119.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_ratings = pd.read_sql("SELECT * FROM Ratings", conn)
C:\U

In [13]:
print(users.columns)


Index(['user_id', 'username', 'birthdate', 'gender', 'height', 'weight',
       'likes', 'dislikes', 'diet_type', 'disease', 'password', 'email',
       'created_at', 'update_at', 'activity_id'],
      dtype='object')


In [24]:
import pandas as pd
import pyodbc

conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=LAPTOP-HJRRM85F;"           # ชื่อ server ของหนู
    "Database=FoodRecommendDB;"
    "Trusted_Connection=yes;"
)

# ดึง rating เฉลี่ยของแต่ละเมนู
query = """
select m.food_id, m.food_name, SUM(r.rating)/COUNT(1) as rating
from Menus m
left join Ratings r on r.food_id = m.food_id
group by m.food_id, m.food_name
order by m.food_id asc
"""
df_rating = pd.read_sql(query, conn)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_1616\581108744.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rating = pd.read_sql(query, conn)


In [25]:
threshold = 7
df_rating['liked'] = df_rating['rating'] >= threshold


In [39]:
def evaluate_hybrid_uids(top_n=10, threshold=7):
    all_precisions, all_recalls = [], []

    for uid in range(1, 54):  # uid 1 ถึง 53
        if uid not in users['user_id'].values:
            print(f"⚠️ ข้าม uid {uid} เพราะไม่มีข้อมูล")
            continue  # ข้าม uid ที่ไม่มีข้อมูล
        recs = hybrid_recommend(uid, top_n=10)
        recommended_ids = recs['food_name'].tolist()

        # เมนูที่ผู้ใช้ชอบจริงจาก rating
        liked_foods = df_rating[df_rating['liked']]['food_name'].tolist()

        # Precision & Recall
        n_rec_k = len(recommended_ids)
        n_rel = len(liked_foods)
        n_rel_and_rec_k = len(set(recommended_ids) & set(liked_foods))

        precision = n_rel_and_rec_k / n_rec_k if n_rec_k else 0
        recall = n_rel_and_rec_k / n_rel if n_rel else 0

        all_precisions.append(precision)
        all_recalls.append(recall)

    print(f"📊 Hybrid Evaluation (Top-{top_n})")
    print(f"   Precision@{top_n}: {np.mean(all_precisions):.4f}")
    print(f"   Recall@{top_n}   : {np.mean(all_recalls):.4f}")
    print(f"   F1@{top_n}       : {2 * np.mean(all_precisions) * np.mean(all_recalls) / (np.mean(all_precisions) + np.mean(all_recalls) + 1e-9):.4f}")

# เรียกใช้
evaluate_hybrid_uids(top_n=10, threshold=1)


⚠️ ข้าม uid 45 เพราะไม่มีข้อมูล
📊 Hybrid Evaluation (Top-10)
   Precision@10: 0.3154
   Recall@10   : 0.0451
   F1@10       : 0.0788


In [46]:
# สร้าง DataFrame ของ liked food โดยใช้ทุก rating >= threshold
threshold = 6
df_rating['liked'] = df_rating['rating'] >= threshold

def evaluate_hybrid_global(top_n=10, threshold=6):
    all_precisions, all_recalls = [], []

    for uid in users['user_id']:
        recs = hybrid_recommend(uid, top_n=top_n)
        recommended_ids = recs['food_name'].tolist()

        # เมนูที่ทุกคนชอบจริง (rating >= threshold จาก Ratings)
        liked_foods = df_rating[df_rating['liked']]['food_name'].tolist()

        # Precision & Recall
        n_rec_k = len(recommended_ids)
        n_rel = len(liked_foods)
        n_rel_and_rec_k = len(set(recommended_ids) & set(liked_foods))

        precision = n_rel_and_rec_k / n_rec_k if n_rec_k else 0
        recall = n_rel_and_rec_k / n_rel if n_rel else 0

        all_precisions.append(precision)
        all_recalls.append(recall)

    print(f"📊 Hybrid Evaluation (Top-{top_n})")
    print(f"   Precision@{top_n}: {np.mean(all_precisions)*100:.2f}%")
    print(f"   Recall@{top_n}   : {np.mean(all_recalls)*100:.2f}%")
    print(f"   F1@{top_n}       : {2 * np.mean(all_precisions) * np.mean(all_recalls) / (np.mean(all_precisions) + np.mean(all_recalls) + 1e-9)*100:.2f}%")

# เรียกใช้
evaluate_hybrid_global(top_n=5, threshold=6)


📊 Hybrid Evaluation (Top-5)
   Precision@5: 43.46%
   Recall@5   : 2.39%
   F1@5       : 4.53%


In [51]:
def health_accuracy(uid, top_n=10):
    recs = hybrid_recommend(uid, top_n=top_n)
    recommended_ids = recs['food_id'].tolist()
    allowed_foods = filter_by_health_constraints(uid, df_final, menu_lookup)['food_id'].tolist()
    n_correct = sum(fid in allowed_foods for fid in recommended_ids)
    return n_correct / len(recommended_ids) if recommended_ids else 0

# คำนวณค่าเฉลี่ยผู้ใช้ทั้งหมด
health_accuracies = [health_accuracy(uid, top_n=10) for uid in users['user_id']]
print("🍏 Health Compliance Accuracy:", np.mean(health_accuracies))


🍏 Health Compliance Accuracy: 1.0


In [56]:
def mood_accuracy(uid, top_n=10):
    recs = hybrid_recommend(uid, top_n=top_n)
    if recs.empty:
        return 0

    user = users[users['user_id'] == uid].iloc[0]
    mood = user['current_mood']

    # ตรวจสอบว่า mood เป็นคอลัมน์ใน df_final หรือ recs
    if mood not in df_final.columns:
        return 0

    # นับจำนวนเมนูที่สอดคล้องกับ mood
    n_correct = 0
    for fid in recs['food_id']:
        if df_final[df_final['food_id'] == fid][mood].values[0] > 0:
            n_correct += 1

    # Accuracy เป็นเปอร์เซนต์
    return n_correct / recs.shape[0] * 100

# คำนวณค่าเฉลี่ยของผู้ใช้ทั้งหมด
mood_accuracies = [mood_accuracy(uid, top_n=10) for uid in users['user_id']]
print(f"💭 Mood Compliance Accuracy: {np.mean(mood_accuracies):.2f}%")


💭 Mood Compliance Accuracy: 42.12%


NameError: name 'python' is not defined